In [2]:
import requests
import json
from bs4 import BeautifulSoup
import config
import datetime
from datetime import timedelta

# Urls con api news y categorías con Beautiful Soup
El resultado es una lista "seleccionadas" con 5 notas por categoría

In [3]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
#'Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/32.0.1667.0 Safari/537.36'

In [4]:
mydate = datetime.date(2018,9, 23) 
fecha = mydate.strftime("%Y-%m-%d")
fecha

'2018-09-23'

In [5]:
newsapi_key = config.newsAPI_KEY

In [6]:
def obtener_urls(fecha):
    seleccionadas = []
    política, economía, elmundo, deportes, espectáculos, sociedad = 0,0,0,0,0,0
    desde= fecha
    hasta= fecha
    #print(fecha)
    fuente= 'la-nacion'
    pagina = 1
    print('Buscando notas')
    #while hasta que se completen los contadores
    while (política < 5) | (economía < 5) | (elmundo < 5) | (deportes < 5) | (espectáculos < 5) | (sociedad < 5):
        url = 'https://newsapi.org/v2/everything?sources=' + fuente + '&from=' + desde + '&to=' + hasta + '&page=' + str(pagina) +'&apiKey=' + newsapi_key
        pagina +=1

        try: 
            response = requests.get(url)
        except:
            print('no anduvo la api')
            break
        data = json.loads(response.text)
        if data['articles'] == []:
            print('no hay más')
            break

        #for todas la noticias de la pagina
        for i, nota in enumerate(data['articles']):
            response = requests.get(nota['url'])
            html_soup = BeautifulSoup(response.text, 'html.parser')
            for x in html_soup.find_all("strong", {"class": "categoria"}):
                categoria = x.find_all('a')[0].text.replace('\r\n', '').strip()

            #print(categoria, end = ', ')

            if (categoria == 'Política') & (política < 5):
                política +=1
                data['articles'][i]['categoria'] = categoria
                seleccionadas.append(data['articles'][i])
            elif (categoria == 'Economía') & (economía <5):
                economía +=1
                data['articles'][i]['categoria'] = categoria
                seleccionadas.append(data['articles'][i])
            elif (categoria == 'El Mundo') & (elmundo <5):
                elmundo +=1
                data['articles'][i]['categoria'] = categoria
                seleccionadas.append(data['articles'][i])
            elif (categoria == 'Deportes') & (deportes <5):
                deportes +=1
                data['articles'][i]['categoria'] = categoria
                seleccionadas.append(data['articles'][i])
            elif (categoria == 'Espectáculos') & (espectáculos <5):
                espectáculos +=1
                data['articles'][i]['categoria'] = categoria
                seleccionadas.append(data['articles'][i])
            elif (categoria == 'Sociedad') & (sociedad <5):
                sociedad +=1
                data['articles'][i]['categoria'] = categoria
                seleccionadas.append(data['articles'][i])

    print('Cantidad de notas encontradas por categoría:', end = ' ' )
    print(política, economía, elmundo, deportes, espectáculos, sociedad)
    return seleccionadas
#que pasa si con todas las paginas que se recuperan de la api no se llega a llenar la cuota? puede pasar?


In [7]:
def obtener_notas(seleccionadas):
    notas = []
    for nota in seleccionadas:
        print ('.', end ='')
        response = requests.get(nota['url'])
        soup = BeautifulSoup(response.text, 'html.parser')
        titular = soup.find('h1', {'class': 'titulo'}).text
        try:
            autor = soup.find('section', {'class': 'autor'}).find('a').text
        except:
            autor = ""
        fecha = nota['publishedAt'][8:10] + '/' + nota['publishedAt'][5:7] + '/' +nota['publishedAt'][:4]
        cuerpo = ''
        for x in soup.find('section', {'id': 'cuerpo'}).find_all('p'):
            cuerpo += x.text
        cuerpo = cuerpo.replace('\n','').replace('\r','')
        comentarios = not bool([True for x in soup.find_all('b') if "Debido a la sensibilidad" in x.text])
        #comentarios = bool(soup.find('div', {'class': 'ver-comentarios'}))
        
        notas.append({'titulo': titular,
                       'autor':autor,
                       'cuerpo':cuerpo,
                       'url': nota['url'],
                       'fecha':fecha,
                       'seccion': nota['categoria'],
                       'abierta_comentarios': comentarios})
    return notas


In [8]:
%%time
notas = []
for dia in range(31):
    fecha = mydate.strftime("%Y-%m-%d")
    print('\n' + fecha)
    seleccionadas = obtener_urls(fecha)
    notas.extend(obtener_notas(seleccionadas))
    mydate += timedelta(days=1)


2018-09-23
Buscando notas
Contadores de categorías: 5 5 5 5 5 5
..............................
2018-09-24
Buscando notas
Contadores de categorías: 5 5 5 5 5 5
..............................
2018-09-25
Buscando notas
Contadores de categorías: 5 5 5 5 5 5
..............................
2018-09-26
Buscando notas
Contadores de categorías: 5 5 5 5 5 5
..............................
2018-09-27
Buscando notas
Contadores de categorías: 5 5 5 5 5 5
..............................
2018-09-28
Buscando notas
Contadores de categorías: 5 5 5 5 5 5
..............................
2018-09-29
Buscando notas
Contadores de categorías: 5 5 5 5 5 5
..............................
2018-09-30
Buscando notas
Contadores de categorías: 5 5 5 5 5 5
..............................
2018-10-01
Buscando notas
Contadores de categorías: 5 5 5 5 5 5
..............................
2018-10-02
Buscando notas
Contadores de categorías: 5 5 5 5 5 5
..............................
2018-10-03
Buscando notas
Contadores de categoría

In [16]:
import pandas as pd
df = pd.DataFrame(notas)

In [17]:
df.fecha= pd.to_datetime(df["fecha"], dayfirst=True)

In [19]:
#pd.DataFrame(notas).sort_values(by = ['fecha', 'seccion']).to_csv('./data/notas_lanacion.csv')
df.sort_values(by = ['fecha', 'seccion']).reset_index(drop= True).to_csv('./data/notas_lanacion.csv')

In [20]:
df.shape

(926, 7)

# Selenium para conseguir comments

In [3]:
import pandas as pd
df = pd.read_csv('./data/notas_lanacion.csv', index_col=0)

In [25]:
#df.abierta_comentarios.value_counts()

In [8]:
#Acá habría que hacerlo para toda la lista, pero pruebo sólo con la primera
#url = seleccionadas[0]['url']
#url = 'https://www.lanacion.com.ar/2181492-hugo-moyano-pedido-detencion-hijo'
#url = 'https://www.lanacion.com.ar/2183250-piden-juicio-oral-daniel-scioli-delitos-corrupcion'


In [77]:
import time
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.chrome.options import Options


path_to_extension = r'C:\Users\p_bag\OneDrive\Documentos\1.17.0_0'
path_to_extension_linux = r'/home/pablo/Documentos/1.17.0_0'
windows = '/Users/p_bag/Downloads/chromedriver_win32/chromedriver.exe'
linux = '/home/pablo/Descargas/chromedriver'

chrome_options = Options()
chrome_options.add_argument('load-extension=' + path_to_extension)
 


In [163]:
def scrapear(urls):
    comentarios= []
    driver = webdriver.Chrome(windows, chrome_options=chrome_options)
    driver.create_options()
    for nota_id, url in zip(urls.index, urls.values):
        #url = df.url[0]
        driver.get(url)

        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        try:
            myElem = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'fyre-stream-more-container')))
            print ("Hay boton")
            button = True
        except TimeoutException:
            print ("No hay boton")
            button = False
        #Que pasa si la nota esta cerrada a comentarios
        while button:
            try:
                driver.find_element_by_class_name('fyre-stream-more-container').click()
            except: 
                button = False
        #todos los comentarios con me gusta, horario y autores,etc
        todo = driver.find_element_by_class_name('fyre-stream-content')
        a= todo.find_elements_by_class_name('fyre-comment-article')
        for item in a:
            try:
                comentarios.append({'texto': item.find_elements_by_class_name("fyre-comment")[0].text,
                                    'usuario' : item.find_elements_by_class_name("fyre-comment-username")[0].text,
                                    'likes' : item.find_elements_by_class_name("fyre-comment-like-count")[0].text,
                                    'fecha' : item.find_elements_by_class_name("fyre-comment-date")[0].text,
                                    'respuestas' : len(item.find_elements_by_class_name('fyre-comment')) -1,
                                    'nota_id': nota_id,
                                    'nota_url': url
                                   })
                print('.', end = '')
            except:
                pass
    return comentarios

In [167]:
corte = df.url.shape[0]//4
lista_urls = []
lista_urls.append(df.url[:corte])
lista_urls.append(df.url[corte:corte*2])
lista_urls.append(df.url[corte*2:corte*3])
lista_urls.append(df.url[corte*3:df.shape[0]])

In [169]:
%%time
from joblib import Parallel, delayed
results = Parallel(n_jobs=-1)(delayed(scrapear)(urls) for urls in lista_urls)

Wall time: 30.6 s


In [170]:
comentarios = results[0] + results[1] + results[2] + results[3]

In [171]:
import pandas as pd
df_comentarios = pd.DataFrame(comentarios)
df_comentarios = df_comentarios.dropna().reset_index(drop = True)
df_comentarios.loc[:, 'respuestas'] = df_comentarios.respuestas.astype(int)
df_comentarios['es_respuesta'] = df_comentarios.texto.str.contains('^@')
df_comentarios

,fecha,likes,nota_id,nota_url,respuestas,texto,usuario,es_respuesta
0,17:43 27/09/2018,,0,https://www.lanacion.com.ar/2175921-cruzeiro-b...,0,Qué se puede esperar de la coimabol gay llina?...,xeneizealpalo,False
1,08:13 27/09/2018,,0,https://www.lanacion.com.ar/2175921-cruzeiro-b...,0,Cruzeiro ahora elimina a Boca!!!,fgattas,False
2,07:18 27/09/2018,2,0,https://www.lanacion.com.ar/2175921-cruzeiro-b...,0,Se hizo justicia. No fue intencional.,BSpinoza,False
3,07:15 27/09/2018,,0,https://www.lanacion.com.ar/2175921-cruzeiro-b...,0,"Fortuito o no el golpe, tambièn se lo puede pe...",sl,False
4,07:08 27/09/2018,1,0,https://www.lanacion.com.ar/2175921-cruzeiro-b...,0,EStudiantes la ultima vez que saliò campeòn ju...,sl,False
5,06:00 27/09/2018,1,0,https://www.lanacion.com.ar/2175921-cruzeiro-b...,1,Primera vez que se deja sin sanción a una expu...,AC1967,False
6,06:45 27/09/2018,1,0,https://www.lanacion.com.ar/2175921-cruzeiro-b...,0,muy buen punto. El VAR no estaba para dilucida...,Madmaster,False
7,05:51 27/09/2018,1,0,https://www.lanacion.com.ar/2175921-cruzeiro-b...,0,el negrito enfiló con su cabeza a matar o mori...,Madmaster,False
8,01:53 27/09/2018,1,0,https://www.lanacion.com.ar/2175921-cruzeiro-b...,0,"Estoy de acuerdo contigo, igual Andrade sale a...",Aldopetti,False
9,01:51 27/09/2018,1,0,https://www.lanacion.com.ar/2175921-cruzeiro-b...,1,"Agresion? Fue un burdo choque, pero choque al ...",Aldopetti,False


In [183]:
df_comentarios.to_csv('./data/commentas_lanacion-4_notas.csv')

#versión sin conteo de respuestas
%%time
textos = todo.find_elements_by_class_name('fyre-comment')
users = todo.find_elements_by_class_name("fyre-comment-username")
fechas = todo.find_elements_by_class_name("fyre-comment-date")
likes = todo.find_elements_by_class_name("fyre-comment-like-count")
usuarios =[x.text for x in users]
textos = [x.text for x in textos]
fechas = [x.text for x in fechas]
likes = [x.text for x in likes]
import pandas  as pd
df = pd.DataFrame({'usuario': usuarios,
                     'comment': textos,
                   'fecha': fechas,
                   'likes': likes
                  })

In [ ]:
#https://www.lanacion.com.ar/2183896-hora-cambios-relacion-estado-iglesia, esta nota no abre automatico los comentarios
#El boton qeu abre lo comentarios es el ver comentarios
<div class="ver-comentarios ver-comentarios">
    <a href="#comentarios">Ver comentarios <span class="ds-none livefyre-commentcount" data-lf-site-id="356483" data-lf-article-id="1353322">218</span></a>
</div>
#en algunas aparece el boton y te lleva a los comentarios que estan abajo,
# en esa el boton dice 218 comments pero te lleva a la cartel de nota cerrada a comentarios
#hay notas cerradas a comentarios que te los muestra y otras que no te lo muestra?

#ejemplo de nota cerrada a comentarios:
#https://www.lanacion.com.ar/688808-cambios-en-un-area-sensible
#hay notas cerradas que tiene comentarios